In [192]:
import numpy as np
import torch
import torch.nn as nn
import torchvision
import matplotlib.pyplot as plt
import torch.optim as optim
import torch.nn.functional as F
from imp import reload
import nice_layers as vnn
reload(vnn)

<module 'nice_layers' from '/Users/davidclark/Projects/VectorizedNets/nice_layers.py'>

In [221]:
#make model

reload(vnn)

model = nn.Sequential(vnn.Conv2d(10, 30, 64, 5, nonneg=False, expanded_input=True),
                      vnn.tReLU(),
                      vnn.AvgPool2d(2),
                      vnn.Conv2d(10, 64, 64, 5, nonneg=True),
                      vnn.tReLU(),
                      vnn.AvgPool2d(2),
                      vnn.Conv2d(10, 64, 120, 5, nonneg=True),
                      vnn.tReLU(),
                      vnn.Flatten(),
                      vnn.Linear(10, 120, 84, nonneg=True),
                      vnn.tReLU(),
                      vnn.Dropout(),
                      vnn.Linear(10, 84, 1, nonneg=True))


In [194]:
#update utilty
def set_model_grads(model, output, labels):
    targets = torch.eye(10)[labels]
    output_error = F.softmax(output, dim=1) - targets
    for i in range(len(model)):
        layer = model[i]
        if type(layer) in (vnn.Conv2d, vnn.Linear):
            if (i == len(model) - 1) or (type(model[i+1]) not in (vnn.ReLU, vnn.tReLU)):
                mask = torch.ones(layer.mask_shape)
            else:
                mask = model[i+1].mask
            layer.set_grad(mask, output_error)
        
#input utility
def expand_input(input, category_dim):
    #input = (batch, channels, width, height)
    batch_size, in_channels = input.shape[:2]
    expanded_input = torch.zeros((batch_size, category_dim, in_channels*category_dim) + input.shape[2:])
    for i in range(category_dim):
        expanded_input[:, i, i*in_channels:(i+1)*in_channels] = input
    return expanded_input
            

In [202]:
#load cifar-10

transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),
                                            torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
train_set = torchvision.datasets.CIFAR10("./data", train=True, download=True, transform=transform)
test_set = torchvision.datasets.CIFAR10("./data", train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_set, batch_size=128, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=128, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [222]:
#loss_fn = nn.CrossEntropyLoss(reduction="mean")
optimizer = optim.SGD(model.parameters(), lr=1e-3, momentum=0)

for epoch_idx in range(1000):
    print("epoch", epoch_idx)
    epoch_loss = 0.
    epoch_correct = 0.
    for batch_idx, (data, labels) in enumerate(train_loader):
        #print(model[3].conv.weight)
        print("batch", batch_idx)
        input = expand_input(data, 10) #.to(0)
        optimizer.zero_grad()
        output = model(input)[..., 0]
        #loss = loss_fn(output.detach(), labels)
        #epoch_loss += loss.item()
        correct = (output.detach().cpu().argmax(dim=1) == labels).float().sum()
        print(correct / 128.)
        epoch_correct += correct
        
        set_model_grads(model, output, labels)
        #print(model[0].conv.weight.grad)
        optimizer.step()
        for layer in model: layer.post_step_callback()

    print(epoch_correct / 50000.)
    #print(epoch_loss / (batch_idx + 1))



epoch 0
batch 0
Instantiated t with shape (10, 64, 28, 28)
Instantiated t with shape (10, 64, 10, 10)
Instantiated t with shape (10, 120, 1, 1)
Instantiated t with shape (10, 84)
tensor(0.0938)
batch 1
tensor(0.0938)
batch 2
tensor(0.1016)
batch 3
tensor(0.1875)
batch 4
tensor(0.1250)
batch 5
tensor(0.1250)
batch 6
tensor(0.1719)
batch 7
tensor(0.1250)
batch 8
tensor(0.1484)
batch 9
tensor(0.1562)
batch 10
tensor(0.1953)
batch 11
tensor(0.2188)
batch 12
tensor(0.1562)
batch 13


KeyboardInterrupt: 

In [212]:
model[3].conv.weight.mean()

tensor(0.0145, grad_fn=<MeanBackward0>)